## Анализ базы данных книжного сервиса при помощи SQL.

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Цель работы:**

Проанализировать базу данных, а именно: 
- посчитать, сколько книг вышло после 1 января 2000 года;
- для каждой книги посчитать количество обзоров и среднюю оценку;
- определить издательство, которое выпустило наибольшее число книг толще 50 страниц, чтобы исключить из анализа брошюры;
- определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
- посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

**Ход работы:**

Импортируем библиотеки, подключимся к базе данных, отобразим cхему базы данных и проведем первичное ознакомление с данными. После этого последовательно решим все необходимые задачи в рамках цели работы, сопровождая их соответствующими выводами. В итоге все завершим общим выводом. 

Таким образом, работа пройдёт в три шага:
1. Ознакомление с данными.
2. Выполнение задач на основе данных.
3. Общий вывод.

⚠️

Импортируем сразу же нужные библиотеки и подключаемся к базе данных, настроив необходимые параметры. Заодно создадим функцию, выводящую результат sql-запроса.

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from IPython.display import Image

db_config = {'user': 'praktikum_student',
    'pwd': 'Sdf4$2;d-d30pp',
    'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
    'port': 6432,
    'db': 'data-analyst-final-project-db'}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
    db_config['pwd'],
    db_config['host'],
    db_config['port'],
    db_config['db'])

engine = create_engine(connection_string, connect_args = {'sslmode':'require'}) 

def execute(query):
    return pd.io.sql.read_sql(query, con = engine)

Отобразим схему базы данных для удобства. 

In [2]:
display(Image(url='https://clck.ru/ray5s', width = 975))

### 1. Ознакомление с данными.

Исследуем таблицы.

---

Посмотрим на первые 5 строк таблицы `books`.

In [3]:
query = '''
SELECT *
FROM books
LIMIT 5
'''

execute(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Посчитаем количество книг.

In [4]:
query = '''
SELECT COUNT(book_id) AS amount
FROM books
'''

execute(query)

,amount
0,1000


Таблица `books` содержит данные о 1000 книгах. Согласно документации:
- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

Посмотрим на первые 5 строк таблицы `authors`.

In [5]:
query = '''
SELECT *
FROM authors
LIMIT 5
'''

execute(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Посчитаем количество авторов.

In [6]:
query = '''
SELECT COUNT(author_id) AS amount
FROM authors
'''

execute(query)

,amount
0,636


Таблица `authors` содержит данные об 636 авторах. Согласно документации:
- `author_id` — идентификатор автора;
- `author` — имя автора.

Посмотрим на первые 5 строк таблицы `publishers`.

In [7]:
query = '''
SELECT *
FROM publishers
LIMIT 5
'''

execute(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Посчитаем количество издательств.

In [8]:
query = '''
SELECT COUNT(publisher_id) AS amount
FROM publishers
'''

execute(query)

,amount
0,340


Таблица `publishers` содержит данные об 340 издательствах. Согласно документации:
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

Посмотрим на первые 5 строк таблицы `ratings`.

In [9]:
query = '''
SELECT *
FROM ratings
LIMIT 5
'''

execute(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Посчитаем количество пользовательских оценок книг.

In [10]:
query = '''
SELECT COUNT(rating_id) AS amount
FROM ratings
'''

execute(query)

,amount
0,6456


Таблица `ratings` содержит данные о 6456 пользовательских оценках книг. Согласно документации:
- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

Посмотрим на первые 5 строк таблицы `reviews`.

In [11]:
query = '''
SELECT *
FROM reviews
LIMIT 5
'''

execute(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Посчитаем количество пользовательских обзоров.

In [12]:
query = '''
SELECT COUNT(review_id) AS amount
FROM reviews
'''

execute(query)

,amount
0,2793


Таблица `reviews` содержит данные о 2793 пользовательских обзорах. Согласно документации:
- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя автора обзора;
- `text` — текст обзора.

**Вывод по шагу:**

Мы исследовали каждую таблицу и выяснили, что:
- таблица `books` содержит данные о 1000 книгах;
- таблица `authors` содержит данные об 636 авторах;
- таблица `publishers` содержит данные о 340 издательствах;
- таблица `ratings` содержит данные о 6456 пользовательских оценках книг;
- таблица `reviews` содержит данные о 2793 пользовательских обзорах.

### 2. Выполнение задач на основе данных.

Последовательно решим все необходимые задачи в рамках цели работы, сопровождая их соответствующими выводами. 

---

#### 2.1 Подсчет книг, вышедших после 1 января 2000 года.

Посчитаем, сколько книг вышло после 1 января 2000 года.

In [13]:
query = '''
SELECT COUNT(book_id) AS amount
FROM books
WHERE publication_date::DATE > '2000-01-01'
'''

execute(query)

,amount
0,819


**Вывод:**

После 1 января 2000 года вышло 819 книг.

#### 2.2 Подсчет количества обзоров и средней оценки для каждой книги.

Для каждой книги посчитаем количество обзоров и среднюю оценку. Затем отсортируем полученные результаты по количеству обзоров и по средней оценке соответственно.

In [14]:
query = '''
SELECT
    books.title,
    COUNT (DISTINCT review_id) AS amount_review,
    AVG(rating) AS average_rating
FROM books
LEFT JOIN reviews ON reviews.book_id = books.book_id
LEFT JOIN ratings ON ratings.book_id = books.book_id
GROUP BY books.title
ORDER BY COUNT(reviews.review_id) DESC
'''

execute(query)

,title,amount_review,average_rating
0,Twilight (Twilight #1),7,3.662500
1,The Hobbit or There and Back Again,6,4.125000
2,The Catcher in the Rye,6,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
...,...,...,...
994,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
995,Essential Tales and Poems,0,4.000000
996,Leonardo's Notebooks,0,4.000000
997,The Natural Way to Draw,0,3.000000


In [15]:
query = '''
SELECT
    books.title,
    COUNT (DISTINCT review_id) AS amount_review,
    AVG(rating) AS average_rating
FROM books
LEFT JOIN reviews ON reviews.book_id = books.book_id
LEFT JOIN ratings ON ratings.book_id = books.book_id
GROUP BY books.title
ORDER BY average_rating DESC
'''

execute(query)

,title,amount_review,average_rating
0,Pop Goes the Weasel (Alex Cross #5),2,5.00
1,Angels Fall,2,5.00
2,Piercing the Darkness (Darkness #2),2,5.00
3,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
4,Neil Gaiman's Neverwhere,2,5.00
...,...,...,...
994,The World Is Flat: A Brief History of the Twen...,3,2.25
995,Junky,2,2.00
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00


Строк стало меньше, похоже, что в названиях книг закрался дубликат. 

In [16]:
query = '''
SELECT
    title,
    COUNT(title)
FROM books
GROUP BY title
HAVING COUNT(title) > 1
'''

execute(query)

,title,count
0,Memoirs of a Geisha,2


Да, это действительно так. 

**Вывод:**

Мы посчитали количество обзоров и среднюю оценку для каждой книги. Видим, что наибольшее количество обзоров у книги `Twilight (Twilight #1)` - 7. Наивысшая средняя оценка - 5, а самая низкая - 1.50. 

#### 2.3 Выявление издательства с наибольшим числом книг.

Определим издательство, которое выпустило наибольшее число книг толще 50 страниц, чтобы исключить из анализа брошюры.


In [17]:
query = '''
SELECT
    publishers.publisher,
    COUNT(book_id)
FROM books INNER JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY count DESC
LIMIT 1
'''

execute(query)

,publisher,count
0,Penguin Books,42


**Вывод:**

Наибольшее количество книг толще 50 страниц выпустило издательство `Penguin Books`, которое точно не занимается брошюрами.

#### 2.4 Выявление автора с самой высокой средней оценкой книг.

Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками.

In [21]:
query = '''
SELECT
    authors.author,
    AVG(ratings.rating) AS average_rating
FROM books
LEFT JOIN ratings ON books.book_id = ratings.book_id
LEFT JOIN authors ON books.author_id = authors.author_id
WHERE
    books.book_id IN (
    SELECT ratings.book_id
    FROM ratings
    GROUP BY ratings.book_id
    HAVING COUNT(ratings.rating_id) > 50)
GROUP BY authors.author
ORDER BY average_rating DESC
LIMIT 1
'''

execute(query)

,author,average_rating
0,J.K. Rowling/Mary GrandPré,4.287097


**Вывод:**

Самая высокая средняя оценка книг, учитывая только книги с 50 и более оценками, у совместных работ Джоан Роулинг и Мэри Грандпре - 4.28. 

#### 2.5 Подсчет среднего количества обзоров от пользователей.

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [20]:
query = '''
SELECT
    AVG(subquery.amount_review) AS amount_average_review
FROM (
    SELECT COUNT(review_id) AS amount_review
        FROM reviews
        WHERE username IN (
            SELECT username
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating) > 50
            )
    GROUP BY username
    ) AS subquery
'''

execute(query)

,amount_average_review
0,24.333333


**Вывод:**

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.33.

**Вывод по шагу:**

Мы последовательно решили все необходимые задачи в рамках цели работы, сопровождая их соответствующими выводами:
- после 1 января 2000 года вышло 819 книг;
- в рамках подсчета количества обзоров и средней оценки для каждой книги выяснили, что наибольшее количество обзоров у книги `Twilight (Twilight #1)` - 7, а наивысшая средняя оценка - 5, самая низкая - 1.50;
- наибольшее количество книг толще 50 страниц выпустило издательство `Penguin Books`, которое точно не занимается брошюрами;
- самая высокая средняя оценка книг, учитывая только книги с 50 и более оценками, у совместных работ Джоан Роулинг и Мэри Грандпре - 4.28;
- среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.33.

### 3. Общий вывод.

Подведем итоги проведенной работы.

---

Мы проанализировали базу данных в соответствии с установленным ходом работы. Импортировали библиотеки, подключились к базе данных, отобразили для удобства схему базы данных и провели первичное ознакомление с данными. Последовательно решили все необходимые задачи в рамках цели работы, сопровождая их соответствующими выводами. 

В базе данных содержатся данные о 1000 книгах, об 636 авторах, об 340 издательствах, о 6456 пользовательских оценках книг и о 2793 пользовательских обзорах. 

По итогу мы выяснили, что:
- 819 книг из 1000 вышло после 1 января 2000 года;
- наибольшее количество обзоров у книги `Twilight (Twilight #1)`;
- наибольшее количество книг толще 50 страниц выпустило издательство `Penguin Books`;
- самая высокая средняя оценка книг, учитывая только книги с 50 и более оценками, у совместных работ Джоан Роулинг и Мэри Грандпре - 4.28;
- среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.33.